# kitti_2_las

Created by Luke McQuade, 25-July-2023.

Additional Contributors: Syeda Zahra Kazmi

Creates a set of georeferenced .las files from a sequence of KITTI .bin files (see https://github.com/PRBonn/semantic-kitti-api).

Designed for use in Google Colab.

## Usage

Upload the Kitti poses.txt, velodyne folder (containing .bin files), and labels folder (containing .label files) into the working directory, set the parameters, and run the remaining script. The .las files will be produced under the 'las' folder.

## Warning

This is a work-in-progress; currently, world orientation (rotation) is not 100% working correctly. There are probably other issues.

## Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/CHANGE_ME/sequences/14'

In [ ]:
import math

# Define the files to process (we will look for files with naming format 000nnn.bin)
num_files = 550
to_process = range(1, num_files + 1, 50)

# Coordinates of origin (world) and next track point, for determining position and orientation
origin_lat, origin_lon = 47.78529034,13.05783747
origin_z = 425 # meters

next_lat, next_lon = 47.78526056,13.05785052
next_z = 425 # meters

# Initial bearing (in radians). Manually specify this if known. Otherwise it is estimated from the above GPS points.
init_bearing_override = math.radians(167.0)

# Scale factor to convert input points to meters
SCALE_FACTOR = 1.0

# Target CRS EPSG (Expected: a projected CRS with meters as units of measurement)
OUT_EPSG = 31255 # (MGI / Austria GK Central)

In [ ]:
%pip install laspy

## Main

In [ ]:
import laspy
import math
import numpy as np
from numpy.linalg import inv
from pyproj import CRS, Transformer
from pathlib import Path


# COULDO: Use this for bearing calculation if using projected coordinates (result has an offset that needs correcting first though)
def calculate_bearing(x1, y1, x2, y2):
    """
    Calculates the bearing between two points (rectangular coordinates).
    Args:
        x1, y1: point 1 coordinates
        x2, y2: point 2 coordinates
    Returns:
        initial_compass_bearing: bearing in radians, clockwise from North.
    """
    dx = x2 - x1
    dy = y2 - y1
    bearing = np.arctan2(dy, dx)
    bearing = (bearing + 2*math.pi) % (2*math.pi)
    return bearing


def calculate_bearing_geo(pointA, pointB):
    """
    Calculates the bearing between two geographic points.
    Args:
        pointA: tuple with latitude and longitude of the first point.
        pointB: tuple with latitude and longitude of the second point.
    Returns:
        initial_compass_bearing: bearing in radians, clockwise from North.
    """
    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])
    diffLong = math.radians(pointB[1] - pointA[1])
    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))
    initial_bearing = math.atan2(x, y)
    initial_bearing = (initial_bearing + 2*math.pi) % (2*math.pi)
    return initial_bearing


def read_poses(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    poses = []
    for line in lines:
        pose = np.array(list(map(float, line.split()))).reshape(3, 4)

        # Complete the affine transform matrix
        pose = np.vstack([pose, [0, 0, 0, 1]])

        poses.append(pose)
    return poses


def process_bin_to_las(bin_path, pose, las_path, origin_proj, bearing, label_path):
    # Correct the bearing for the world transform matrix and pose matrix axis differences
    bearing = bearing - (math.pi / 2)

    x1, y1, z1 = origin_proj
    # Load points from .bin file
    points_orig = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)

    # Convert points to meters
    points = points_orig[:,:3] * SCALE_FACTOR  # 4th channel doesn't need conversion/transforming

    # Create homogeneous coordinates for points
    points_hom = np.ones((points.shape[0], 4))
    points_hom[:, :3] = points

    # Create a homogeneous transformation matrix for world coordinates
    homogeneous_transformation_world = np.array([
        [np.cos(-bearing), -np.sin(-bearing), 0, x1],
        [np.sin(-bearing),  np.cos(-bearing), 0, y1],
        [0, 0, 1, z1],
        [0, 0, 0, 1]
    ])

    # Create the combined transformation matrix
    combined_transformation = homogeneous_transformation_world @ pose

    # Apply the combined transformation to the points
    world_points_hom = (combined_transformation @ points_hom.T).T

    # Extract the world coordinates (discard the homogeneous coordinate)
    world_points = world_points_hom[:,:3]

    # Load labels
    labels = np.fromfile(label_path, dtype=np.int32)

    # Create las
    las = laspy.create(point_format=2, file_version='1.4')
    las.header.add_crs(CRS.from_user_input(OUT_EPSG))
    las.x = world_points[:, 0]
    las.y = world_points[:, 1]
    las.z = world_points[:, 2]
    las.intensity = points_orig[:, 3]
    las.user_data = labels # if this fails, instance id is too high and we need to find a different solution
    las.write(las_path)

poses_path = "poses.txt"

poses = read_poses(poses_path)

# Transform origin to target CRS
t = Transformer.from_crs(f"EPSG:4326", f"EPSG:{OUT_EPSG}", always_xy=True)
origin_x, origin_y = t.transform(origin_lon, origin_lat)
next_x, next_y = t.transform(next_lon, next_lat)

if init_bearing_override is None:
  # TODO: Fix this - leads to odd rotations later.
  # init_bearing = calculate_bearing(origin_x, origin_y, next_x, next_y)
  init_bearing = calculate_bearing_geo((origin_lat, origin_lon), (next_lat, next_lon))
else:
  init_bearing = init_bearing_override

bin_paths = [f"velodyne/{n:06}.bin" for n in to_process]
las_paths = [f"las/{n:06}.las" for n in to_process]
poses_to_process = [poses[n-1] for n in to_process]
label_paths = [f"labels/{n:06}.label" for n in to_process]

Path("las/").mkdir(exist_ok=True)

for bin_path, las_path, pose, label_path in zip(bin_paths, las_paths, poses_to_process, label_paths):
  print(f"Processing: {bin_path}, output: {las_path}")
  process_bin_to_las(bin_path, pose, las_path, (origin_x, origin_y, origin_z), init_bearing, label_path)

Processing: velodyne/000001.bin, output: las/000001.las
Processing: velodyne/000051.bin, output: las/000051.las
Processing: velodyne/000101.bin, output: las/000101.las
Processing: velodyne/000151.bin, output: las/000151.las
Processing: velodyne/000201.bin, output: las/000201.las
Processing: velodyne/000251.bin, output: las/000251.las
Processing: velodyne/000301.bin, output: las/000301.las
Processing: velodyne/000351.bin, output: las/000351.las
Processing: velodyne/000401.bin, output: las/000401.las
Processing: velodyne/000451.bin, output: las/000451.las
Processing: velodyne/000501.bin, output: las/000501.las
